In [ ]:
import pandas as pd
import folium

In [6]:
df_full = pd.read_excel('Olympics_dataset_Final_product.xlsx')

In [81]:
df = df_full[df_full['season'].isin(['Summer Olympics', 'Winter Olympics'])]

#df = df.drop(columns=['Original name', 'Nick/petnames', 'Title(s)', 'unknown_id', 'used_name', 'full_name', 'birth_date_Olympedia',
#                      'birth_country_Olympedia', 'birth_region_2024', 'Roles', 'Affiliations', 'NOC', 'Other names', 'Name order',
#                      'age_Olympedia', 'death_date', 'birth_country_full_Olympedia', 'Born', 'Died', 'birth_date', 'birth_country_full (2024 imported too)'])


df_swimming = df[df['sport'] == 'Swimming']

In [88]:
df_swimming

,unknown_id,id,used_name,full_name,gender,height_in_cm,weight_in_kg,birth_date,birth_date_Olympedia,age,...,Died,NOC,Affiliations,Nick/petnames,Title(s),Other names,Nationality,Original name,Name order,medal_points
24,540.0,183.0,D. Frangopoulos,D. Frangopoulos,Male,NaN,NaN,17 April 1878,NaN,18.0,...,NaN,Greece,Panakhaikos Syllogos Patron,NaN,NaN,NaN,NaN,Δ.•Φρανγόπουλοσ,NaN,0
25,541.0,183.0,D. Frangopoulos,D. Frangopoulos,Male,NaN,NaN,17 April 1878,NaN,18.0,...,NaN,Greece,Panakhaikos Syllogos Patron,NaN,NaN,NaN,NaN,Δ.•Φρανγόπουλοσ,NaN,0
26,542.0,183.0,D. Frangopoulos,D. Frangopoulos,Male,NaN,NaN,17 April 1878,NaN,18.0,...,NaN,Greece,Panakhaikos Syllogos Patron,NaN,NaN,NaN,NaN,Δ.•Φρανγόπουλοσ,NaN,0
93,37301.0,21169.0,Otto Herschmann,Otto Herschmann,Male,NaN,NaN,4 January 1877,4 January 1877,19.0,...,"17 June 1942 in Sobibór, Lubelskie (POL)",Austria,"1.W.A.S.C., Wien (AUT) / Wiener AC, Wien (AUT)",NaN,NaN,NaN,NaN,NaN,NaN,2
94,37303.0,21169.0,Otto Herschmann,Otto Herschmann,Male,NaN,NaN,4 January 1877,4 January 1877,19.0,...,"17 June 1942 in Sobibór, Lubelskie (POL)",Austria,"1.W.A.S.C., Wien (AUT) / Wiener AC, Wien (AUT)",NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315647,295139.0,144950.0,Han Da-Gyeong,Han Da-Gyeong,Female,NaN,NaN,31 January 2000,31 January 2000,20.0,...,NaN,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN,Oriental,0
315648,295140.0,144950.0,Han Da-Gyeong,Han Da-Gyeong,Female,NaN,NaN,31 January 2000,31 January 2000,20.0,...,NaN,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN,Oriental,0
315653,295141.0,144950.0,Han Da-Gyeong,Han Da-Gyeong,Female,NaN,NaN,31 January 2000,31 January 2000,20.0,...,NaN,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN,Oriental,0
315674,NaN,1907187.0,Sven Schwarz,NaN,Male,NaN,NaN,31 January 2002,NaN,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
medal_to_points = {
    'Gold': 3,
    'Silver': 2,
    'Bronze': 1,    
    None: 0,
}

df_swimming['medal_points'] = df_swimming['medal'].apply(lambda x: medal_to_points.get(x, 0))

/var/folders/_r/ncd8qcvx7b3dtm5wvv799ksw0000gn/T/ipykernel_67254/2788005408.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swimming['medal_points'] = df_swimming['medal'].apply(lambda x: medal_to_points.get(x, 0))


In [ ]:
def medalists_on_map(df, only_gold=False, name_map_suffix="map", after_year=None):
    if only_gold:
        df_medalists = df[df['medal'] == 'Gold']
    else:
        df_medalists = df_swimming[df_swimming['medal'].notna()]
    
    if after_year is not None:
        df_medalists = df_medalists[df_medalists['year'] > after_year]

    df_medalists = df_medalists.dropna(subset=['latitude', 'longitude'])
    m = folium.Map(location=[df_medalists['latitude'].mean(), df_medalists['longitude'].mean()], zoom_start=2)

    for _, row in df_medalists.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"{row['full_name']} - {row['medal']}",
            color='gold' if row['medal'] == 'Gold' else 'silver' if row['medal'] == 'Silver' else 'brown',
            fill=True,
            fill_opacity=0.7
        ).add_to(m)

    m.save(f'medalists_map_{name_map_suffix}.html')



medalists_on_map(df_swimming, only_gold=False, name_map_suffix="only_gold_swimmers", after_year=None)

In [ ]:
cols = ['id', 'gender', 'age', 'birth_country_full', 'year', 'team', 'event', 'medal_points', 'place', 'Population', 'Gini', ]


gdp_cols = [col for col in df_swimming.columns if 'GDP' in col]
df_analysis = df_swimming[cols + gdp_cols] 
df_analysis = df_analysis[df_analysis['year'] > 1924]
df_analysis['event'] = df_analysis['event'].apply(lambda x: x.replace(', Women', "").replace(', Men', ""))
df_analysis['relay'] = df_analysis['event'].apply(lambda x: 1 if 'Relay' in x else 0)


#df_analysis['Gini'] = df_analysis['Gini'].astype(float)

#df_analysis['gender'] = df_analysis['gender'].apply(lambda x: 1 if x == 'Male' else 0)
#df_analysis['distance'] = df['event'].apply(lambda x: x.split(' metres ')[0])


#number of individual participants
#proportion of #participants to population
#is the country a host?
#distance of event
#style of event

ValueError: could not convert string to float: 'no_data'

In [109]:
df_analysis['event'].unique()#.groupby(by=['team', ''])

array(['4 × 200 metres Freestyle Relay', '100 metres Backstroke',
       '100 metres Freestyle', '400 metres Freestyle',
       '200 metres Breaststroke', '4 × 100 metres Freestyle Relay',
       '1,500 metres Freestyle', '4 × 100 metres Medley Relay',
       '100 metres Butterfly', '200 metres Butterfly',
       '100 metres Breaststroke', '200 metres Individual Medley',
       '400 metres Individual Medley', '200 metres Freestyle',
       '200 metres Backstroke', '50 metres Freestyle',
       '800 metres Freestyle', '4 x 100 metres Freestyle Relay',
       '4 x 100 metres Medley Relay', '4 x 200 metres Freestyle Relay',
       'Mixed 4 x 100 metres Medley Relay', 'Relay Only Athlete',
       '4 × 100 metres Medley Relay, Mixed'], dtype=object)

In [110]:
df_analysis

,id,gender,age,birth_country_full,year,team,event,medal_points,place,Population,Gini,"GDP per capita, PPP (constant 2021 international $)","GDP total, PPP (constant 2021 international $)",GDP per capita growth (%),relay
5955,49775.0,1,26.0,Poland,1936,POL,4 × 200 metres Freestyle Relay,0,2.0,28592912,21.7,2964.654332109682,84768100428.4309,1.01728201099764,1
7120,47918.0,1,NaN,NaN,1928,GER,100 metres Backstroke,0,4.0,65297668,31.4,7462.0006142946,487251238728.004,1.03772683858643,0
8492,46584.0,1,19.0,Estonia,1936,EST,100 metres Backstroke,0,4.0,1121015,38.3,5560.968940380624,6233929596.70078,1.06194690265486,0
8493,46585.0,1,18.0,Estonia,1936,EST,100 metres Freestyle,0,4.0,1121015,38.3,5560.968940380624,6233929596.70078,1.06194690265486,0
8499,46585.0,1,18.0,Estonia,1936,EST,400 metres Freestyle,0,NaN,1121015,38.3,5560.968940380624,6233929596.70078,1.06194690265486,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315647,144950.0,0,20.0,NaN,2020,KOR,800 metres Freestyle,0,28.0,51858482,31.6,46506.93304216386,2411778950042.26,0.991537877622007,0
315648,144950.0,0,20.0,NaN,2020,KOR,"1,500 metres Freestyle",0,28.0,51858482,31.6,46506.93304216386,2411778950042.26,0.991537877622007,0
315653,144950.0,0,20.0,NaN,2020,KOR,4 × 200 metres Freestyle Relay,0,14.0,51858482,31.6,46506.93304216386,2411778950042.26,0.991537877622007,1
315674,1907187.0,1,22.0,NaN,2024,GER,"1,500 metres Freestyle",0,NaN,84552242,31.7,61981.37894190373,5315370899343.56,1.01544046494074,0


In [113]:
df_analysis.groupby(by=['team', 'year', 'gender']).agg(avg_age = ('age', 'mean'), 
                                                        gini = ('Gini', 'mean'))

TypeError: agg function failed [how->mean,dtype->object]